In [ ]:
import torch
import torch.nn as nn
from torchvision.transforms import transforms
from torch.utils.data import DataLoader
from torch.optim import Adam
import torchvision
import glob

In [ ]:
classes = ['f','j','k','l','m','n','o','x','y','z']

transformer = transforms.Compose([
    transforms.ToTensor(),
    transforms.RandomHorizontalFlip(),
    transforms.Normalize([0.5,0.5,0.5],
                        [0.5,0.5,0.5])
])

train_path = 'data_letters/train'
test_path = 'data_letters/test'

trainloader = DataLoader(
    torchvision.datasets.ImageFolder(train_path, transform=transformer),
    batch_size = 256,
    shuffle = True
)

testloader = DataLoader(
    torchvision.datasets.ImageFolder(test_path, transform=transformer),
    batch_size = 256,
    shuffle = True
)

In [ ]:
class Task1Net(nn.Module):
    def __init__(self, num_classes=10):
        super(Task1Net, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=12, kernel_size=3, stride=1, padding=1)
        self.bn1 = nn.BatchNorm2d(num_features=12)
        self.relu1 = nn.ReLU()
        self.pool = nn.MaxPool2d(kernel_size=2)
        self.dropout = nn.Dropout(0.5)
        self.conv2 = nn.Conv2d(in_channels=12, out_channels=20, kernel_size=3, stride=1, padding=1)
        self.relu2 = nn.ReLU()
        self.conv3 = nn.Conv2d(in_channels=20, out_channels=32, kernel_size=3, stride=1, padding=1)
        self.bn3 = nn.BatchNorm2d(num_features=32)
        self.relu3 = nn.ReLU()
        self.fc = nn.Linear(in_features=32*32*32, out_features=num_classes)
        
    def forward(self, input):
        output = self.conv1(input)
        output = self.bn1(output)
        output = self.relu1(output)
        output = self.pool(output)
        output = self.dropout(output)
        output = self.conv2(output)
        output = self.relu2(output)
        output = self.conv3(output)
        output = self.bn3(output)
        output = self.relu3(output)
        output = output.view(-1, 32*32*32)
        output = self.fc(output)
        return output

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = torchvision.models.resnet34(pretrained=True)
model.to(device)

inchannel = model.fc.in_features
model.fc = nn.Linear(inchannel, 10)

optimizer = Adam(model.parameters(), lr=0.0001)
loss_function = nn.CrossEntropyLoss()

num_epoches = 10

train_count = len(glob.glob(train_path + '/**/*.bmp'))
test_count = len(glob.glob(test_path + '/**/*.bmp'))

best_accuracy = 0.0
for epoch in range(num_epoches):
    model.train()
    train_accuracy = 0.0
    train_loss = 0.0
    for i, (images, labels) in enumerate(trainloader):
        if torch.cuda.is_available():
            images = Variables(images.cuda())
            labels = Variables(labels.cuda())
        
        optimizer.zero_grad()
        outputs = model(images)
        loss = loss_function(outputs, labels)
        loss.backward()
        optimizer.step()
        
        train_loss += loss.cpu().data * images.size(0)
        _, prediction = torch.max(outputs.data,1)
        train_accuracy += int(torch.sum(prediction == labels.data))
    
    train_accuracy = train_accuracy / train_count
    train_loss = train_loss/train_count
    
    model.eval()
    test_accuracy = 0.0
    with torch.no_grad():
        for i, (images, labels) in enumerate(testloader):
            if torch.cuda.is_available():
                images = Variables(images.cuda())
                labels = Variables(images.cuda())
            outputs = model(images)
            _,prediction = torch.max(outputs.data,1)
            test_accuracy += int(torch.sum(prediction==labels.data))

        test_accuracy = test_accuracy / test_count
        print('Epoch: ' + str(epoch) + ' Train Loss: '+str(train_loss)+' Train Accuracy: '+str(train_accuracy)+ ' Test Accuracy: '+str(test_accuracy))

        if test_accuracy > best_accuracy:
            torch.save(model.state_dict(), 'model/task1.model')
            best_accuracy = test_accuracy